In [ ]:
import pydicom
from PIL import Image
import matplotlib.pyplot as plt
from pydicom import dcmread
from pydicom.data import get_testdata_file
import os
import numpy as np
import cv2



In [ ]:
base_dir = "/home/shihab/Desktop/Machine Learning/Dataset/3dImageDataset/minData/147-Tahera Sultana. Dr_Nurul_Amin"


In [6]:
import os
import numpy as np
# Directory containing patient folders
base_dir = "/home/shihab/Desktop/Machine Learning/Dataset/3dImageDataset/minData"

# List to store all pixel data
all_patient_data = []

# Iterate through patient folders
patients = os.listdir(base_dir)
for patient in patients:
    patient_dir = os.path.join(base_dir, patient)
    dicom_files = [f for f in os.listdir(patient_dir) if f.endswith('.dcm')]

    for dicom_file in dicom_files:
        file_path = os.path.join(patient_dir, dicom_file)
        
        # Read DICOM file
        dicom_data = pydicom.dcmread(file_path)
        
        # Extract pixel array and normalize it
        pixel_array = dicom_data.pixel_array
        normalized_pixels = (pixel_array - np.min(pixel_array)) / (np.max(pixel_array) - np.min(pixel_array))
        
        # Flatten the array and append to the list
        all_patient_data.append(pixel_array.flatten())

# Convert the list to a NumPy array
dt = np.array(all_patient_data)

# Print the shape of the resulting array
print("Shape of the combined data:", dt.shape)


Shape of the combined data: (1350, 360000)


In [19]:
import pydicom
import os
import numpy as np

def load_dicom_series(base_dir):
    slices = []
    for filename in sorted(os.listdir(base_dir)):
        if filename.endswith(".dcm"):
            filepath = os.path.join(base_dir, filename)
            slices.append(pydicom.dcmread(filepath))
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))  # Sort by slice position
    volume = np.stack([s.pixel_array for s in slices], axis=-1)  # Create 3D volume
    return volume


In [20]:
def normalize(volume):
    volume = volume - np.min(volume)
    volume = volume / np.max(volume)
    return volume


In [26]:
import SimpleITK as sitk

def resample_volume(volume, new_spacing=(1.0, 1.0, 1.0)):
    sitk_volume = sitk.GetImageFromArray(volume)
    original_spacing = sitk_volume.GetSpacing()
    original_size = sitk_volume.GetSize()
    new_size = [
        int(np.round(original_size[i] * (original_spacing[i] / new_spacing[i])))
        for i in range(3)
    ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampled_volume = resampler.Execute(sitk_volume)
    return sitk.GetArrayFromImage(resampled_volume)


In [27]:
def extract_slices(volume):
    return [volume[:, :, i] for i in range(volume.shape[2])]


In [29]:
!pip install numpy pydicom opencv-python scikit-image matplotlib SimpleITK nibabel tensorflow keras pytorch torchvision monai

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def unet_model(input_size=(128, 128, 1)):
    inputs = Input(input_size)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    # Add more layers similarly...
    up = UpSampling2D((2, 2))(p1)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(up)
    return Model(inputs, outputs)

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 57.4 kB/s eta 0:00:0000:0100:07
     ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 216.3/615.3 MB 23.2 kB/s eta 4:46:54
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/lib/python3/dist-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/lib/python3/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.10/http/client.py", line 466, in read
    s = self.fp.read(amt)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.1

ModuleNotFoundError: No module named 'tensorflow'